# TensorRT - DO NOT USE

Start with an understanding of your model and the TensorFlow versions - which leads to the CUDA versions.   If on SageMaker - this will not be super flexible.   AWS may be lagging on TF version.

IMPORTANT NOTE 
- use deb (dpkg) on Ubuntu
- use rpm (yum) on AWS Linux

I learned the hardway - if you use the *.run files, which looks easier - it may not register the packages correctly

### READ THE DOCUMENTATION CAREFULLY
https://docs.nvidia.com/deeplearning/sdk/tensorrt-install-guide/index.html
TensorFlow 1.14 required  
CUDA 10.0 or 10.2 required -- 10.1 NOT supported (and just guess what version we have!)
So problem - how do you get SageMaker to use CUDA 10.0 an the 410 driver?  


### TensorRT version 7.0 
### CUDA 10.0 w/ driver 410 -OR- CUDA 10.1 & 418
The point here is, you must have this consistent.   I think TensorFlow 1.14 will work fine with CUDA 10.0 or 10.1 and TensorFlow 1.15 will wants CUDA 10.1.     On SageMaker, you have less flexibility because AWS will typically be a TF version behind.   On your physical machine, you may be on the current version.

In [ ]:
# this will show you your CUDA version and driver version
# note that nvcc is NOT installed

# the symbol, cuda, is pointing to cuda-10.0
! ls /usr/local -la

# BUT, nvidia-smi says 10.1
! nvidia-smi

# I learned the hardway, that means CUDA 10.1 is installed - so install TensorRT for 10.1
# you can ignore the symbol (cuda) for now

In [ ]:
import tensorflow as tf
print (tf.__version__)

In [ ]:
# verify cuBLAS (part of the CUDA toolkit) is installed
! cat /usr/local/cuda/include/cublas.h | grep CUBLAS
! nvcc --version

### 20200113 - on SageMaker p2
Tensorflow 1.14.0  
CUDA 10.1  
NVIDIA Driver 418.87  

So we 


In [ ]:
import os

In [ ]:
# Globals

PROJECT_DIR = os.getcwd()
BUCKET = 's3://cfaanalyticsresearch-sagemaker'

SAGEMAKER_DIR = os.path.join(PROJECT_DIR, "..")


## Get the TensorRT software

https://developer.nvidia.com/nvidia-tensorrt-7x-download

928 MB This will not go well if you are on CFA network !!!!
  - leverage a Workspace if possible (super fast)
  - use a curl command (I didn't figure this out) - downloading directly to SageMaker should be fast
  - use S3 (I put current packages on the cfaanalyticsresearch-sagemaker/software
  - Don't download over CFA/COWS to your computer then upload!! it will fail
For SageMaker:  get a RPM package (20200113) CentOS/RedHat 7 CUDA 10.0  
For Ubuntu:


In [ ]:
# SageMaker
# - upload the RPM package to the SageMaker directory; 

# look in S3
! aws s3 ls {BUCKET}/software/

# download to SageMaker directory
# - you only have to do this once
# - but you'll need to install every time you reboot


# TensorRT
! aws s3 cp {BUCKET}/software/ {SAGEMAKER_DIR} --exclude "*.*" --include "nv-tensorrt-repo-rhel7-cuda10.0-trt7*.rpm" --recursive 

## Installation Instructions
https://docs.nvidia.com/deeplearning/sdk/tensorrt-archived/tensorrt-700/tensorrt-install-guide/index.html  
    
Read the instructions carefully  
Pre-requesites - regardless of your Linux 
- CUDA obviously - 10.1 is the target version at this time (driven by your TensorFlow version)
- cuBLAS - doesn't look like cuBLAS is on SageMaker (might be on your local machine)  cuBLAS is included in the CUDA Toolkit

Check your environment - this is based on using TensorFlow 1.14 - which wants CUDA 10.1  

cuBLAS
driver - 410.48
CUDA version


### SageMaker Installation
Read 4.2 for RPM installation  
Jupyter won't keep the variable names correctly so,  cut paste this into the terminal.  
- open a Jupyter terminal
- go to SageMaker directory

c
Pre-requisites
Error: Package: libnvinfer7-7.0.0-1.cuda10.0.x86_64 (nv-tensorrt-cuda10.0-trt7.0.0.11-ga-20191216)
           Requires: cuda-cublas-10-0
Error: Package: libnvinfer-devel-7.0.0-1.cuda10.0.x86_64 (nv-tensorrt-cuda10.0-trt7.0.0.11-ga-20191216)
           Requires: cuda-cublas-dev-10-0
Error: Package: libnvinfer-devel-7.0.0-1.cuda10.0.x86_64 (nv-tensorrt-cuda10.0-trt7.0.0.11-ga-20191216)
           Requires: cuda-cudart-dev-10-0
Error: Package: libnvinfer7-7.0.0-1.cuda10.0.x86_64 (nv-tensorrt-cuda10.0-trt7.0.0.11-ga-20191216)
           Requires: cuda-cudart-10-0
           
`wget http://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda_10.2.89_440.33.01_linux.run`
`sudo sh cuda_10.2.89_440.33.01_linux.run`         


CUDA Toolkit (to get cuBLAS-10.0)
https://developer.nvidia.com/cuda-10.0-download-archive  
I put 10.1 on S3 and you downloaded it above - so you just have to install.  
`sudo rpm -i cuda-repo-rhel7-10-1-local-10.1.105-418.39-1.0-1.x86_64.rpm`  
`sudo yum clean all`  
`sudo yum install cuda`  

Get the tag value from the current version (file) you downloaded 

`tag="cuda10.0-trt7.0.0.11-ga-20191216"`  
`sudo rpm -Uvh nv-tensorrt-repo-rhel7-${tag}-1-1.x86_64.rpm`  
`sudo yum clean expire-cache`  

In [ ]:
### Ubuntu Installation

## Convert the Model
### Step 1 - convert the checkpoint to TensorFlow Lite

See TrainModel_Step3_TrainingJob.ipynb

#### Convert (Trained) Model Checkpoint to a tflite Model


ALSO NOTE:  the script is named convert_checkpoint_to_edge_tflite.sh  
Well... the name is no longer totally accurate
- I took this from the original Coral TPU tutorial
- Another step is required for compiling for the EdgeTPU (not really relevant here since we are confined to AWS where there is no TPU -- so we skip that stuff)
- And, I added a step that converts the checkpoint to a TENSORFLOW frozen graph 
  - note that this generates a frozen_inference_graph.pb
  - it ALSO generates a saved model graph.pb  
  THESE (frozen graph & saved model) are NOT the SAME!!  
  
https://stackoverflow.com/questions/46547319/error-when-parsing-graph-def-from-string

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/exporting_models.md

In [ ]:
! ls code/model

In [ ]:
# WAKE UP - make sure the checkpoint num == hyperparameters/num_train_steps
# convert checkpoint is a task script - located in the tasks/ directory
os.chdir("tasks")  
! ./convert_checkpoint_to_edgetpu_tflite.sh --checkpoint_num {NUM_TRAINING_STEPS} --pipeline_config {PIPELINE_CONFIG}


In [ ]:
# https://www.tensorflow.org/lite/convert/python_api#using_the_interpreter_from_a_model_file_

# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model('code/model')
tflite_model = converter.convert()
